In [ ]:
!pip install -U sentence-transformers
!pip install transformers

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



import os
import sys
import tensorflow as tf
import torch
from torch.utils.data import DataLoader


from tqdm import tqdm

import math


In [ ]:
from transformers import BertModel,BertTokenizer

In [ ]:
from sentence_transformers import models,losses
from sentence_transformers import SentenceTransformer

In [ ]:
covid_ert=BertModel.from_pretrained("deepset/covid_bert_base")
try:
    os.makedirs("/kaggle/working/covid_ert")
except:
    pass
covid_ert.save_pretrained("/kaggle/working/covid_ert")

!wget https://s3.amazonaws.com/models.huggingface.co/bert/deepset/covid_bert_base/vocab.txt -P /kaggle/working/covid_ert
## vocabfile for covid_ert, downloading the beert model

In [ ]:
from sentence_transformers import models,losses
from sentence_transformers import SentenceTransformer
word_embedding_model = models.Transformer("/kaggle/working/covid_ert")

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(768,
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=True,
                               pooling_mode_max_tokens=True)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model]) ## generating biobert sentence embeddings (mean pooling of sentence embedding vectors)

In [ ]:
df_mancon=pd.read_csv("/kaggle/input/mancon-corpus-cleaned/manconcorpus_sent_pairs.tsv",sep="\t").rename(columns={"guid":"pairID",
                                                                                                                 "text_a":"sentence1",
                                                                                                                 "text_b":"sentence2"})

In [ ]:
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

In [ ]:
class NLIDataReader(object):
    def __init__(self,dataframe):
        self.df=dataframe.copy()
    def get_examples(self,max_examples=0):
        s1=self.df["sentence1"].values
        s2=self.df["sentence2"].values
        labels=self.df["label"].values
        guid=self.df["pairID"].values
        examples = []
        for sentence_a, sentence_b, label, guid_id in zip(s1, s2, labels, guid):

            examples.append(InputExample(guid=guid_id, texts=[sentence_a, sentence_b], label=self.map_label(label)))

            if 0 < max_examples <= len(examples):
                break

        return examples
    
    
    
    @staticmethod
    def get_labels():
        return {"contradiction": 0, "entailment": 1, "neutral": 2}

    def get_num_labels(self):
        return len(self.get_labels())

    def map_label(self, label):
        return self.get_labels()[label.strip().lower()]

In [ ]:
def get_labels():
    return {"contradiction": 0, "entailment": 1, "neutral": 2}

In [ ]:
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer,SentencesDataset

In [ ]:
df_mancontrain,df_mancontest=train_test_split(df_mancon,test_size=0.2)
df_mancontest,df_manconval=train_test_split(df_mancontest,test_size=0.5)

In [ ]:
nli_reader=NLIDataReader(df_mancontrain)
train_num_labels = nli_reader.get_num_labels()
batch_size=16

train_data = SentencesDataset(nli_reader.get_examples(), model=model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=train_num_labels)

val_nli_reader=NLIDataReader(df_manconval)
dev_data = SentencesDataset(val_nli_reader.get_examples(), model=model)


dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)

In [ ]:
model_save_path="/kaggle/working/model_covid_ert_mancon/"
num_epochs=1
warmup_steps = math.ceil(len(train_dataloader) * num_epochs / batch_size * 0.1) #10% of train data for warm-up

model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )

In [ ]:
model = SentenceTransformer(model_save_path)

test_nli_reader=NLIDataReader(df_mancontest)
test_data = SentencesDataset(test_nli_reader.get_examples(), model=model)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
evaluator = EmbeddingSimilarityEvaluator(test_dataloader)

In [ ]:
model.evaluate(evaluator)

In [ ]:
df_mancontrain['sentence1_embedding']=df_mancontrain['sentence1'].apply(lambda x: np.array(model.encode([x])[0]))
df_mancontrain['sentence2_embedding']=df_mancontrain['sentence2'].apply(lambda x: np.array(model.encode([x])[0]))
df_mancontrain['vector']=df_mancontrain.apply(lambda x: np.array(x.sentence1_embedding.tolist()+x.sentence2_embedding.tolist()),axis=1)
df_mancontrain['target']=df_mancontrain['label'].apply(lambda x:get_labels()[x])

In [ ]:
sample_df=pd.read_excel("/kaggle/input/annotation-drug-similarity/drug_individual_claims_similarity_annotated.xlsx")
contradiction_df=sample_df[["paper1_cord_uid","paper2_cord_uid","text1","text2","annotation"]]

contradiction_df=contradiction_df[~contradiction_df.annotation.isna()]
contradiction_df.rename(columns={"text1":"sentence1","text2":"sentence2","annotation":'label'},inplace=True)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

from sklearn.linear_model import LogisticRegression

In [ ]:
train_data=df_mancontrain['vector'].values
train_data = np.vstack(train_data[:][:])
train_target=df_mancontrain['target'].values

In [ ]:
logreg=LogisticRegression()
logreg.fit(train_data,train_target)

In [ ]:
df_mancontest['sentence1_embedding']=df_mancontest['sentence1'].apply(lambda x: np.array(model.encode([x])[0]))
df_mancontest['sentence2_embedding']=df_mancontest['sentence2'].apply(lambda x: np.array(model.encode([x])[0]))
df_mancontest['vector']=df_mancontest.apply(lambda x: np.array(x.sentence1_embedding.tolist()+x.sentence2_embedding.tolist()),axis=1)
df_mancontest['target']=df_mancontest['label'].apply(lambda x:get_labels()[x])

test_data=df_mancontest['vector'].values
test_data = np.vstack(test_data[:][:])
test_target=df_mancontest['target'].values

In [ ]:
test_result=logreg.predict(test_data)
print(classification_report(test_target,test_result))

In [ ]:
contradiction_df['sentence1_embedding']=contradiction_df['sentence1'].apply(lambda x: np.array(model.encode([x])[0]))
contradiction_df['sentence2_embedding']=contradiction_df['sentence2'].apply(lambda x: np.array(model.encode([x])[0]))
contradiction_df['vector']=contradiction_df.apply(lambda x: np.array(x.sentence1_embedding.tolist()+x.sentence2_embedding.tolist()),axis=1)
contradiction_df['target']=contradiction_df['label'].apply(lambda x:get_labels()[x])

test_data_final=contradiction_df['vector'].values
test_data_final = np.vstack(test_data_final[:][:])
test_target=contradiction_df['target'].values

In [ ]:
test_result=logreg.predict(test_data_final)
print(classification_report(test_target,test_result))

In [ ]:
contradiction_df['predicted']=test_result

In [ ]:
contradiction_df.to_csv("Result_annotation.csv",index=False)